In [2]:
import scipy.io
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import math
import soundfile as sf
import os
from pydub import AudioSegment

In [3]:
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\sample"
audio_names = os.listdir(audio_path)
audio_names

['20190621_010000.WAV',
 '20190621_020000.WAV',
 '20190621_030000.WAV',
 '20190621_040000.WAV',
 '20190621_050000.WAV',
 '20190621_060000.WAV',
 '20190621_070000.WAV',
 '20190621_080000.WAV',
 '20190621_090000.WAV']

# Audio extraction

In [4]:
bird_tags = scipy.io.loadmat('Bird_tags_Train.mat')["Bird_tags"]
coords = {}
inverted_coords = {}
for elem in bird_tags:
    tag = elem[0][0][0][0][0]
    file_name = elem[0][0][0][1][0]
    bbox = elem[0][0][0][3][:4]
    start_time = math.floor(min(bbox[:, 0]))
    end_time = math.floor(max(bbox[:, 0]))
    duration = end_time - start_time
    if tag not in coords:
        coords[tag] = []
    if file_name not in inverted_coords:
        inverted_coords[file_name] = []
    coords[tag].append({"file_name": file_name, "bbox": bbox.tolist(), "start_time": start_time, "duration": duration})
    inverted_coords[file_name].append({"tag": tag, "start_time": start_time, "duration": duration})


In [5]:
# generate json file of coords
import json
with open("utils/coords.json", "w") as f:
    json.dump(coords, f)
with open("utils/inverted_coords.json", "w") as f:
    json.dump(inverted_coords, f)

In [6]:
# con una finestra di 3 secondi, andiamo a individuare gli uccelli
# presenti in ciascuna finestra, e li mettiamo in un insieme
# number_chunks = int(librosa.get_duration(y=y, sr=sr) / 3) 
number_chunks = 200
y_true_raw = []
for audio_name in audio_names:
    audio_chunks = [set() for i in range(number_chunks)]
    for bird in inverted_coords[audio_name]:
        for i in range(bird['duration']):
            position = (bird['start_time'] + i) // 3
            if position < number_chunks:
                audio_chunks[position].add(bird['tag'].replace("_", " "))
    y_true_raw.append(audio_chunks)

In [7]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from birdnetlib.batch import DirectoryAnalyzer
from pprint import pprint
from datetime import datetime

all_recordings = []
def on_analyze_complete(recording):
    print("Analyzing ", recording.path)
    # pprint(recording.detections)
    all_recordings.append(recording)

def on_error(recording, error):
    print("An exception occurred: {}".format(error))
    print(recording.path)

In [8]:
print("Starting Analyzer")

custom_model_path = "TestClassifier.tflite"
custom_labels_path = "custom_species_list.txt"
analyzer = Analyzer(
    classifier_labels_path=custom_labels_path, 
    classifier_model_path=custom_model_path
)
analyzer

Starting Analyzer
load_custom_models
Custom model loaded.
loading custom classifier labels
Labels loaded.
load model False
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [9]:
print("Starting Watcher")
directory = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\sample"
batch = DirectoryAnalyzer(
    directory,
    analyzers=[analyzer],
    lon=12.28458,
    lat=46.31664,
    date=datetime(year=2019, month=6, day=21),
    min_conf=0.1,
)

batch.on_analyze_complete = on_analyze_complete
batch.on_error = on_error
batch.process()

Starting Watcher
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_010000.WAV
Analyzing  D:\Giacomo\Tovanella-20241110T120546Z-001\sample\20190621_010000.WAV
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_020000.WAV
Analyzing  D:\Giacomo\Tovanella-20241110T120546Z-001\sample\20190621_020000.WAV
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_030000.WAV
Analyzing  D:\Giacomo\Tovanella-20241110T120546Z-001\sample\20190621_030000.WAV
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_040000.WAV
Analyzing  D:\Giacomo\Tovanella-20241110T120546Z-001\sample\20190621_040000.WAV
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_050000.WAV
Analyzing  D:\Giacomo\Tovanella-20241110T120546Z-001\sample\20190621_050000.WAV
read_audio_data
read_audio_data: complete, read  200 chunks.
analyze_recording 20190621_

In [10]:
y_pred_raw = []
for recording in all_recordings:
    model_preds = recording.detections
    model_chunks_labels = [[] for i in range(number_chunks)]
    recording_chunks = [set() for i in range(number_chunks)]
    for pred in model_preds:
        duration = int(pred['end_time'] - pred['start_time'])
        start_time = int(pred['start_time'])
        position = start_time // 3
        model_chunks_labels[position].append({'tag': pred['scientific_name'], 'conf': pred['confidence']})
        recording_chunks[position].add(pred['scientific_name'].replace("_", " "))
    y_pred_raw.append(recording_chunks)

In [25]:
y_pred_raw

[[{'Aegithalos caudatus'},
  set(),
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  set(),
  {'Nucifraga caryocatactes'},
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  set(),
  {'Nucifraga caryocatactes'},
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  set(),
  {'Aegithalos caudatus', 'Nucifraga caryocatactes'},
  set(),
  {'Nucifraga caryocatactes'},
  set(),
  {'Aegithalos caudatus'},
  {'Aegithalos caudatus'},
  set(),
  set(),
  set(),
  set(),
  {'Aegithalos caudatus'},
  set(),
  set(),
  set(),
  set(),
  set(),
  {'Nucifraga caryocatactes'},
  set(),
  set(),
  {'Nucifraga caryocatactes'},
  {'Aegithalos ca

# Global Evaluation
chunks are evaluated globally, there's no dependence to the original audio

In [11]:
all_true_labels = set()
all_true_labels = set([key.replace("_", " ") for key in coords.keys()])
all_pred_labels = [specie.split("_")[0] for specie in analyzer.custom_species_list]
all_pred_labels = set(all_pred_labels)
all_labels = all_pred_labels.union(all_true_labels)

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([all_labels])

y_true = []
for i, record in enumerate(y_true_raw):
    y_true.append(mlb.transform(y_true_raw[i]))

y_pred = []
for i, record in enumerate(y_pred_raw):
    y_pred.append(mlb.transform(y_pred_raw[i]))


In [13]:
y_pred

[array([[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
   

In [14]:
from sklearn.metrics import hamming_loss

hamming_loss(y_true[3], y_pred[3])

0.0475

In [18]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_true[3], y_pred[3], average="micro")

(0.0, 0.0, 0.0, None)

In [24]:
from sklearn.metrics import multilabel_confusion_matrix

cms = multilabel_confusion_matrix(y_true[3], y_pred[3])
for label, cm in zip(all_labels, cms):
    print(label, cm)

Phylloscopus trochilus [[135  65]
 [  0   0]]
Dryocopus martius [[200   0]
 [  0   0]]
Aegithalos caudatus [[200   0]
 [  0   0]]
Prunella modularis [[200   0]
 [  0   0]]
Muscicapa striata [[200   0]
 [  0   0]]
Chloris chloris [[200   0]
 [  0   0]]
Regulus regulus [[200   0]
 [  0   0]]
Cyanistes caeruleus [[200   0]
 [  0   0]]
Aeroplane [[200   0]
 [  0   0]]
Corvus cornix [[200   0]
 [  0   0]]
Poecile palustris [[200   0]
 [  0   0]]
Emberiza citrinella [[200   0]
 [  0   0]]
Turdus viscivorus [[200   0]
 [  0   0]]
Columba palumbus [[200   0]
 [  0   0]]
Pecking [[196   4]
 [  0   0]]
Nucifraga caryocatactes [[200   0]
 [  0   0]]
Periparus ater [[200   0]
 [  0   0]]
Carduelis carduelis [[200   0]
 [  0   0]]
Certhia familiaris [[200   0]
 [  0   0]]
Cuculus canorus [[200   0]
 [  0   0]]
Loxia curvirostra [[200   0]
 [  0   0]]
Coleus monedula [[200   0]
 [  0   0]]
Anthus trivialis [[200   0]
 [  0   0]]
Chainsaw [[200   0]
 [  0   0]]
Phylloscopus collybita [[200   0]
 [  0